In [1]:
from requests_html import HTMLSession
import numpy as np
from lxml import html
session = HTMLSession()

In [2]:
# traverse poster gallery to get detail page links
import sys
import hashlib
url = 'https://www.palestineposterproject.org/wellspring/palestinian-nationalist-artistsagencies?page='
baseUrl = 'https://www.palestineposterproject.org/'

posters = []
for i in range(0, 135):
    
    r = session.get(url + str(i))
    posterPrev = r.html.find('.poster-grid .poster-grid-views-row .poster-grid-views-column')
    for t in posterPrev:
        title =  t.find('.views-field-title .field-content a')[0].text        
        hashedInit = hashlib.md5(title.encode())
        fileid = str(int(hashedInit.hexdigest(), 16))[0:12]

        posters.append({
            'Id': fileid
            , 'Title': title
            ,'Link': baseUrl + t.find('.views-field-title .field-content a')[0].attrs.get('href')
            ,'Thumb':  t.find('.views-field-field-poster-image .field-content a img')[0].attrs.get('src')
        })

print(len(posters))

5400


In [37]:
# create poster title map

posterTitleMap = {}
for p in posters:
    posterTitleMap[p.get('Title')] = p;

In [51]:
# UPDATE POSTER IDS, SOME WEIRD ONE TIME SHIT

# save image metadata

# IMPORT IMAGES
import json

imgMap = {}
with open('./Documents/OmarGit/parsons-machine-learning/final/data-mapping/posterAttrClean - Copy.json') as json_file:
    imgMap = json.load(json_file)

titleMapOld = {}
for key in imgMap:
    obj = imgMap.get(key)
    titleMapOld[obj.get('Title')] = obj;
    
idMapUpdated = {}    
for title in titleMapOld:
    obj = titleMapOld[title]
    newId = posterTitleMap[title].get('Id')
    obj['Id'] = newId;
    idMapUpdated[newId] = obj;
    
with open('./Documents/OmarGit/parsons-machine-learning/final/data-mapping/posterAttrIdClean.json', 'w') as outfile:
    json.dump(idMapUpdated, outfile)
print('done')

In [57]:
imgMapId = {}
with open('./Documents/OmarGit/parsons-machine-learning/final/data-mapping/posterAttrIdClean.json') as json_file:
    imgMapId = json.load(json_file)


In [59]:
def cleanLabel(l):
    if l == 'Artist/Designer/Photographer':
        return 'Artist'
    if l == 'Source/Credit/Provenance':
        return 'Source'
    else:
        return l

In [18]:
# # testing, one detail obj
# from bs4 import BeautifulSoup as bs
# dpage = session.get(posters[0].get('Link'))
# parsed = bs(dpage.html.find('#page')[0].html, 'html.parser')
# parsed.select_one('figure a').get('href')

In [19]:
from bs4 import BeautifulSoup as bs

def getAttributes(detailPage):

    parsed = bs(detailPage.html.find('#page')[0].html, 'html.parser')
    fieldNum = len(parsed.select('.field-label'))
    attributes = {}
    
    attributes['FullImage'] = parsed.select_one('figure a').get('href')

    for index in range(0, fieldNum):
        label = parsed.select('.field-label')[index].text.split(':')[0]
        label = cleanLabel(label)       
        values = parsed.select('.field-label~div')[index].select('div > div')
        valueText = []
        for val in values:
            valueText.append(val.text)  
        attributes[label] = valueText
        
    return attributes

In [25]:
posterAttributes = []

i = 0;
for p in posters:
    if (i % 100 == 0):
        print(i)
    detailPage = session.get(p['Link'])
    detailAttributes = getAttributes(detailPage)
    p.update(detailAttributes)
    posterAttributes.append(p)
    i = i+1

print('done')

0


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
# # IMPORT IMAGES
# import json

# imgMap = {}
# with open('./Documents/OmarGit/parsons-machine-learning/final/poster-assets/posterAttr.json') as json_file:
#     imgMap = json.load(json_file)
# print(len(imgMap))

In [60]:
imgMapId.get('338673932409')

{'Id': '338673932409',
 'Title': 'Great Good News For the Honored People of Haifa',
 'Link': 'https://www.palestineposterproject.org//poster/great-good-news-for-the-honored-people-of-haifa',
 'Thumb': '469109849428',
 'FullImage': 'https://www.palestineposterproject.org/sites/default/files/rohikhamas_pppa.jpg',
 'Translation/Interpretation/Caption Text': None,
 'Date': 1933,
 'Artist': ['Research in Progress '],
 'Source': ['Sent in by Salim Tamari'],
 'Iconography': ['Music/Dance/Instrument/Song/Lyrics'],
 'Special Collection': ['Sightings',
  'Musa Budeiri Collection',
  'Visible Storage'],
 'Language': ['Arabic'],
 'Features': [0.7226654291152954, 0.21094733476638794],
 'GridOrder': 4239}

In [61]:
# get images from url + save locally

from PIL import Image

numFailed = 0
i = 0
for poster in imgMapId:
    if i % 200 == 0:
        print(i)
    i = i +1
    poster_id = poster.get('Id')
    img_url = poster.get('Thumb')
    
    if (img_url):
        file_ext = './Documents/OmarGit/parsons-machine-learning/final/poster-assets-thumb/' + poster_id + '.jpg'
        try:
            img = Image.open(session.get(img_url, stream=True).raw)
            img.convert('RGB').save(file_ext)
        except Exception:
            numFailed += 1
            print('exception', img_url)
 
print('failed', numFailed)
print('success', i-numFailed)

0


AttributeError: 'str' object has no attribute 'get'

In [62]:
imgMapId.get('338673932409')

{'Id': '338673932409',
 'Title': 'Great Good News For the Honored People of Haifa',
 'Link': 'https://www.palestineposterproject.org//poster/great-good-news-for-the-honored-people-of-haifa',
 'Thumb': '469109849428',
 'FullImage': 'https://www.palestineposterproject.org/sites/default/files/rohikhamas_pppa.jpg',
 'Translation/Interpretation/Caption Text': None,
 'Date': 1933,
 'Artist': ['Research in Progress '],
 'Source': ['Sent in by Salim Tamari'],
 'Iconography': ['Music/Dance/Instrument/Song/Lyrics'],
 'Special Collection': ['Sightings',
  'Musa Budeiri Collection',
  'Visible Storage'],
 'Language': ['Arabic'],
 'Features': [0.7226654291152954, 0.21094733476638794],
 'GridOrder': 4239}

In [65]:
# get images from url + save locally

from PIL import Image

numFailed = 0
i = 0
for poster in imgMapId:
    if i % 200 == 0:
        print(i)
    i = i +1
    poster = imgMapId.get(poster)
    poster_id = poster.get('Id')
    img_url = poster.get('FullImage')
    
    if (img_url):
        file_ext = './Documents/OmarGit/parsons-machine-learning/final/poster-assets/' + poster_id + '.jpg'
        try:
            img = Image.open(session.get(img_url, stream=True).raw)
            img.convert('RGB').save(file_ext)
        except Exception:
            numFailed += 1
            print('exception', img_url)
 
print('failed', numFailed)
print('success', i-numFailed)

0
200
400
600
800
1000
exception https://www.palestineposterproject.org/sites/default/files/0216_pppa.jpg
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
exception https://www.palestineposterproject.org/sites/default/files/parc_treasury_pppa.jpg
3800
exception https://www.palestineposterproject.org/sites/default/files/kasaba_pppa.jpg
4000
4200
4400
4600
4800
5000
failed 3
success 5139


In [6]:
import hashlib
str2hash = 'me mama'
result = hashlib.md5(str2hash.encode())
str(int(result.hexdigest(), 16))[0:12]

# m.update("some string")
# str(int(m.hexdigest(), 16))[0:12]

'244969085143'